# dyld 过程学习记录

去年就看过 sunnyxx 大牛的经典文章 [iOS 程序 main 函数之前发生了什么](https://blog.sunnyxx.com/2014/08/30/objc-pre-main/) 但是一直没有时间去验证源码，在逆向学习的过程中，之前学习了 **Mach-O** 二进制格式的基础以及 **fishhook** 的原理，相信对 **Mach-O** 格式已经有了整体的把握，所以再来看这篇文章，记录一些学习心得。

## pre-main 应用启动预备阶段

iOS 中用到的所有系统 *framework* 都是动态链接的，所谓动态链接，就是在 App 运行时再完成链接这一动作。而 **dyld (The Dynamic Link Editor)** 便是用来处理动态链接的准备工作。

学习这一部分我们先了解一下 Apple 对于 App 启动流程分成了两个阶段 `pre-main` 阶段和 `main()` 阶段。

### pre-main 阶段

包括：`1. 加载应用的可执行文件` -> `2. 加载动态链接器 dyld` -> `3. dyld 递归加载所有的 dylib`

### main() 阶段

包括：`1. dyld 调用 main() 方法` -> `2. UIApplicationMain 调用` -> `3. applicationWillFinishLaunching 调用` -> `4. didFinishLaunchingWithOptions 调用`

在很多关于 APM 技术或者启动优化的文章中，都有对于启动的时间统计方法。在之前的 WWDC 上，Apple 已经为我们提供了一种测量方法，在 *Xcode* 中打开 `Edit scheme`，选择 `Run` 一栏中的 `Arguments` 选项卡，在 `Environment Variables` 中加入 `DYLD_PRINT_STATISTICS` 环境变量，并赋值为 `1`，如下图：

![pre-main 启动时间](image/ch01/ch01_1.jpeg)

在运行 App 的时候发现 Terminal 输出了以下 Logs:

```shell
Total pre-main time: 506.96 milliseconds (100.0%)
         dylib loading time: 398.48 milliseconds (78.6%)
        rebase/binding time:  19.42 milliseconds (3.8%)
            ObjC setup time:  44.05 milliseconds (8.6%)
           initializer time:  44.91 milliseconds (8.8%)
           slowest intializers :
             libSystem.B.dylib :  12.02 milliseconds (2.3%)
    libMainThreadChecker.dylib :  14.48 milliseconds (2.8%)
```

此时它的输出与 `pre-main` 阶段的耗时一一对应。下面从耗时开始来讲述这个阶段中的作用以及主角 `dyld`。

## dyld 过程

基本上一个大公司的工程对于启动时间的优化所有的手段都会集中在优化 `dyld` 过程中。`dyld` 是一个用户态的进程，切不属于 Kernel 的一部分，而是作为一个单独的开源项目（但也是 Darwin 的一部分）由 Apple 进行维护。

在新的进程开始时，Kernel 设置用户模式的入口方法为 `__dyld_start`，在开源的 `dyld` 源码中对应的是 `dyldStartup.s` [文件](https://github.com/opensource-apple/dyld/blob/master/src/dyldStartup.s)，在注释中作者写到该文件与 GCC 中 `crt0.o` 很像，通过它来连接到可执行文件中从而启动程序。

```Assembly
__dyld_start:
	mov 	x28, sp
	and     sp, x28, #~15		// 强制堆栈 16 位对齐
	mov	x0, #0
	mov	x1, #0
	stp	x1, x0, [sp, #-16]!	// 入栈操作
	mov	fp, sp			// set up fp to point to terminating frame
	sub	sp, sp, #16             // make room for local variables
	ldr     x0, [x28]		// get app's mh into x0
 	ldr     x1, [x28, #8]           // get argc into x1 (kernel passes 32-bit int argc as 64-bits on stack to keep alignment)
	add     x2, x28, #16		// get argv into x2
	adrp	x4,___dso_handle@page
	add 	x4,x4,___dso_handle@pageoff // get dyld's mh in to x4
	adrp	x3,__dso_static@page
	ldr 	x3,[x3,__dso_static@pageoff] // get unslid start of dyld
	sub 	x3,x4,x3		// x3 now has slide of dyld
	mov	x5,sp                   // x5 has &startGlue
	
	// call dyldbootstrap::start(app_mh, argc, argv, slide, dyld_mh, &startGlue)
	bl	__ZN13dyldbootstrap5startEPK12macho_headeriPPKclS2_Pm // 这里做了 BL 跳转命令
	mov	x16,x0                  // 在 x16 寄存器中保存入口地址
	ldr     x1, [sp]
	cmp	x1, #0
	b.ne	Lnew
```

这里我们可以看出 `__dyld_start` 的执行先处理了一下内存堆栈，最终会调用 `dyldbootstrap::start` 方法。这里最重要的参数无非是第一个 `macho_header` 的一个指针。在进入改方法后，直接观察程序堆栈，发现 `dyldbootstrap::start` 执行后会立即触发 `dyld::_main` 方法。这个过程我们可通过对一个 *iOS/macOS* 项目增加一个**`_objc_init` 符号断点** 进行验证：

![_objc_init堆栈](image/ch01/ch01_2.jpeg)

在 `dyldbootstrap::start` 会直接看到以下堆栈代码，于是便调起了 `dyld::_main` 方法。

```Assembly
0x1000063cf <+448>: callq  0x10000a5ef               ; dyld::_main(macho_header const*, unsigned long, int, char const**, char const**, char const**, unsigned long*)
0x1000063d4 <+453>: addq   $0x38, %rsp
```

`dyld` 加载动态库的代码就是从 `dyld::_main` 开始执行的，在这个方法中将会完成递归加载动态库过程，我们从 dyld 源码 *360.18* 版本进行解析：

```c
// _main 是 dyld 的入口方法。从 Kernel 中加载到 dyld 并跳转到 __dyld_start
// __dyld_start 会创建一些寄存器并调起该方法。
//
// Return 返回 __dyld_start 后进而跳转至 Target 程序的 main 方法

// Params

uintptr_t _main(const macho_header* mainExecutableMH, uintptr_t mainExecutableSlide, 
		int argc, const char* argv[], const char* envp[], const char* apple[], 
		uintptr_t* startGlue)
```

一些参数的含义：

* **mainExecutableMH**：传入的 Mach-O 头
* **mainExecutableSlide**: 个人的猜测是 Target 程序的 `main` 方法入口对应其 Mach-O Header 的一个地址偏移
* **argc**：参数个数
* **argv**：参数选项
* **envp**：环境变量集
* **apple**：针对于 Mach-O 的一些上下文扩展

